
# B.8. Pre-Calibration Tutorial for Coupled Human-Water Systems Modeling

This notebook demonstrates a precalibration workflow for coupled human-water systems modeling. 

To avoid long runtimes, we provide pre-generated simulation results so users can focus on experiencing the precalibration workflow and analyzing the outcomes. The simulation code is provided at the end of this notebook, allowing users to run the model on their own local machine if preferred.

The code block below is to setup the environment.

In [2]:
# Install and load packages

# Install packages if not already installed
#!pip install pathnavigator

import pathnavigator

root_dir = rf"{pathnavigator.__path__[0]}/.."
pn = pathnavigator.create(root_dir)

## B.8.1 Why Pre-Calibration?

Before jumping into "how" to do the pre-calibration, let's start by answering "why" we may need this technique.

Pre-calibration is a pragmatic, flexible, and low-cost alternative, especially in the following situations:

- Models are computationally expensive

If a model take a long time to run, even for just a minute, the computational cost may not be feasible for Markov Chain Monte Carlo (MCMC) or full Bayesian inference as the number of model evaluations (in the scale of tens of thousands to millions) in those more formal uncertainty quantification techniques is much higher than what pre-calibration is required.

- Full probabilistic treatment is unjustified or infeasible

Many complex simulation models, especially those involving human decision-making or agent-based behaviors, lack an explicit or tractable likelihood function. This makes it difficult or impossible to formally apply Bayesian inference methods, which rely on the likelihood to update prior beliefs. For example, agent-based models may include discrete, rule-based decisions and path dependencies that don’t lend themselves to a mathematically smooth or differentiable likelihood surface.

- The problem space is large and uncertain

Many models are structurally complex and include numerous parameters, often with limited observational data to constrain them, which is often the case for human-water systems modeling. A full probabilistic approach would require defining priors and exploring joint distributions over potentially under-informed parameters, which could lead to unidentifiable or non-informative posteriors. The equifinality phenonminon, where multiple model configurations result in similar outcomes, add another layer of complexity and difficulty in uncertainty quantification.

- Decision-makers may require rapid insights

In some cases, decision-makers may require rapid insights or robustness testing over a range of plausible futures, rather than full posterior distributions. Pre-calibration offers a practical compromise that it helps filter implausible parameter sets based on observed behavior, enabling focused exploration while avoiding overcommitment to unjustified assumptions.

## B.8.2 Pre-Calibration in Coupled Human-Water Systems Modeling

Moving toward the "how" question, we want to learn how the conventional parameterization works for constructing a coupled human-water model then gradually look into what pre-calibration strategies we could use to reveal the uncertainty of the model.

Given the complexity of coupled human-water model, often time a single best model configuration is picked from the component-wise model calibration. The uncertainty of this type of the model is then solely built from its stochatic nature in the model design. From the model configuration perspective, this can be view as a deteministic best model which ignore the uncertainty that could be brought by parameter uncertainty.

**Why parameter uncertainty matter in human-water systems modeling context?**

Assuming we have a coupled model consists of a semi-distributed hydrological model and an agent-based model (ABM) where each agent may interact with different subbasins as well as other agent. Now, we could calibrate the hydrological model and ABM seperately using the available data. Suppose we pick a hydrological model that performed the best on a choosen error metric at the basin outlet and coupled with an ABM to evaluate a water conservation policy, what we ignore is the potential equifinal parameter configurations. Those equifinal hydrological models may have similar performance at the outlet but may vary in their subbasin-level hydrological responses. Remembering that agents are interacting with subbasins, the policy evaluation results derive from the coupled model simulation in this case may have significant uncertainty. 

**Pre-Calibration Strategies**

Instead of using single best model configuration, this tutorial will walk you through how to reveal the parameter uncertainty through pre-calibration technique. Specifically, we want to compare two pre-calibration strategies: one is to conduct a component-wise pre-calibration the other is pre-calibration over a coupled model.

If both stratgies achieve similar uncertainty information, component-wise pre-calibration maybe a more computationally efficienct workflow given a parameter sampling space is smaller compared to the full coupled model.

Now, let's start the tutorial by introducing a coupled human-water model, CoFlow, that we will used as an example.

## B.8.3 CoFlow Model

CoFlow is a stylized human water system model inspired by Yakima River Basin (YRB) in Washinton State. CoFlow is built by HydroCNHS python package (Lin et al., 2022) that comprises a semi-distributed hydrological model and an ABM to describe five irrigation district annual diversion behaviors under the basin-wide water conservation program. The water conservation program are represented by gradually increase summer flow target at gauge G. In this tutorial, 

![](.images/yrb.jpg)

### Hydrological model
Each subbasin's runoff responses are simulated using Generalized Watershed Loading Functions (GWLF; Haith & Shoemaker, 1987; Tung & Haith, 1995) where the routing process are computed by the Lohmann routing model (Lohmann et al., 1998; Wi et al., 2015). The parameters are shown below.

Table 1. Hydrological model parameters
| Model              | Sub-model        | Parameter name                                 | Unit     | Code |
|--------------------|------------------|--------------------------------------------------|----------|------|
| Hydrological model | GWLF             | Curve number                                     | –        | $CN2$  |
|                    |                  | Interception coefficient                         | –        | $IS$   |
|                    |                  | Recession coefficient                            | –        | $Res$  |
|                    |                  | Deep seepage coefficient                         | –        | $Sep$  |
|                    |                  | Baseflow coefficient                             | –        | $α$    |
|                    |                  | Percolation coefficient                          | –        | $β$    |
|                    |                  | Available/soil water capacity                    | Cm       | $Ur$   |
|                    |                  | Degree-day coefficient for snowmelt              | cm/°C    | $Df$   |
|                    |                  | Land cover coefficient                           | –        | $Kc$   |
|                    | Lohmann routing  | Subbasin unit hydrograph shape parameter         | –        | $Gs$   |
|                    |                  | Subbasin unit hydrograph rate parameter          | –        | $Gr$   |
|                    |                  | Wave velocity in the linearized Saint–Venant eq. | m/s      | $Ve$  |
|                    |                  | Diffusivity in the linearized Saint–Venant eq.   | m²/s     | $Di$   |

### ABM
Each agent in ABM simulate one of the five irrigation district's diversion behavior. It will make annual diversion request decision in March every year. In the designed ABM used in this study, we assume agent can learn from the long-term envioronmental changes based on the flow target violation feedback and adapt to short-term shocks using the forecast water availability represented by the precipitation during the winter and spring period given that the streamflow is main supply by the snow melt.

Essentially, diversion agents'  have learning, adaptive, and drought response elements involve in their decision-making procedure. For detailed calculation please refer to (Lin & Yang, 2022). Here we would like to provide you a high level understanding on agent decision-making procedure.

#### Learning

The diversion request decision procedure start with adjusting diversion request references ($Div_{req,ref}$) from the flow deviation of the flow target capturing agent's learning ability.

$$
Div_{\text{req,ref},y} = Div_{\text{req,ref},y-1} + V_{\text{avg},y} \times \gamma
$$

$$
V_{\text{avg},y} = \frac{1}{10} \sum_{i=1}^{10} V_{y-i}
$$

where $ \gamma $ is a learning rate, subscript $ y $ denotes the year, and $ V_{\text{avg}} $ is the average strength value, with a 10-year rolling window indicating the magnitude and learning direction (e.g., increase $ Div_{\text{req,ref},y} $ if $ V_{\text{avg},y} $ is positive, or decrease $ Div_{\text{req,ref},y} $ if $ V_{\text{avg},y} $ is negative).

This 10-year rolling window also creates learning momentum in the learning direction of $ Div_{\text{req,ref}} $, where $ V_{\text{avg}} $ needs several (consecutive) counter events (e.g., $ V = 1 $ when $ V_{\text{avg}} < 0 $ or $ V = -1 $ when $ V_{\text{avg}} > 0 $) to reverse its sign (i.e., learning direction of $ Div_{\text{req,ref}} $).

The events are denoted as $V$ and calculated by:

$$
V_y = 
\begin{cases} 
1 & \text{if } |De_y| > L_u \text{ and } De_y > 0 \\
-1 & \text{if } |De_y| < L_l \text{ and } De_y < 0 \\
0 & \text{otherwise}
\end{cases}
$$

$$
De = Q_{789} - Q_{\text{target}} 
$$

where $ L_u $ and $ L_l $ are the upper and lower flow deviation thresholds, respectively. These two parameters control how sensitive an agent is to wet or dry hydrological conditions. $ De $ is the deviation of the average flow from July to September ($ Q_{789} $) relative to the flow target ($ Q_{\text{target}} $).

![](images\learning.jpg)

#### Adapting
Then, the updated $ Div_{\text{req,ref},y} $ will be further adjusted according to available water, which is approximated by the precipitation from Nov to Jun, through a quadratic function as shown below.

$$
Div_{\text{req,mu},y} = Div_{\text{req,ref},y} + a \times P_{11{-}6,y}^2 + b \times P_{11{-}6,y} + c
$$

#### Drought response
If a drought occurs defined as $ P_{11{-}6,y} < 315 mm $, the adaptative behavior above will be replaced by a fixed prorated ratio ($ R $).

$$
Div_{\text{req,mu},y} = Div_{\text{req,ref},y} \times R
$$

Finally, the interaction among farmers are described by covariance matrix fitted by historical diversion data.

$$
Div_{\text{req},y} = Div_{\text{req,mu},y} + Rn \times Sig
$$

where $ Rn $ is a random number draw from the multivariate normal distribution scaled by $ sig $. The requested diversion ($Div_{\text{req}}$) is disaggregated into a daily scale based on historical monthly reduction proportions to continue the simulation. 

The return flow ($Q_{r,d}$) is computed from the actual diversion ($Div_d$) as shown below:

$$
Q_{r,d} = frout(R_f \times Div_d) 
$$

where $frout(\cdot)$ represents the routing process within the subbasin of the returned outlet, $R_f$ is the return flow factor, and the subscript $d$ denotes the day.


The table below summarize the parameters involved in ABM calculation.

Table 2. ABM parameters
| Parameter Description                          | Unit         | Symbol     |
|-----------------------------------------------|--------------|------------|
| Return flow factor                             | –            | $R_f$      |
| Upper flow deviation threshold                 | m³/s         | $L_U$      |
| Lower flow deviation threshold                 | m³/s         | $L_L$      |
| Learning rate                                  | –            | $\gamma$   |
| Standard deviation modifier                    | m³/s         | $Sig$      |
| Prorated ratio                                 | –            | $R$        |
| Quadratic coefficient of quadratic model       | m³/s/cm²     | $a$      |
| Slope of quadratic model                       | m³/s/cm      | $b$      |
| Intercept of quadratic model                   | m³/s         | $c$      |

## B.8.4 Error and Managment Metrices

## B.8.5 Latin Hyper Cube (LHC) Sampling on the Selected Parameters 

## B.8.6 Component-Wise Pre-Calibration V.S. Single Best Model Configuration

## B.8.7 Comparison between Component-Wise and As-a-Whole Pre-Calibration

## B.8.8 (Optional) Simulation Code



## B.8.2 Impacts of Pre-Calibration Strategies on Uncertainty Analysis in Coupled Human-Water Modeling

- Show the findings of our experiment first
- Brings out the experiment designs => Three pre-calibration strategies

## B.8.3 CoFlow Model: A Coupled Flow and Behavior Modeling Framework Inspired by the Yakima River Basin

- Introducing the modeling setup and management problem in the YRB

## B.8.4 Error Metrics V.S. Management Metrices

- Introducing the metrics that we used in this analysis and their implications

## B.8.5 Identifying Plausible Hydrological Models

## B.8.6 Identifying Plausible Agent-Based Models

## B.8.7 Pre-calibration with Hydrological Model Ensembles x Agent-Based Model Ensembles

## B.8.8 Pre-calibration with Full CoFlow Model

## B.8.9 Comparison over Pre-Calibration Strategies and Their Policy Implications

## B.8.10 (Optional) Simulation Code 
